In [1]:
!pip install nltk

You are using pip version 8.1.2, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import nltk

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
from nltk.corpus import wordnet
syns = wordnet.synsets("tEST")
print (syns[0].name())

trial.n.02


In [22]:
from nltk.corpus import wordnet
syns = wordnet.synsets("Executed")
print (syns[0].lemmas)

<bound method Synset.lemmas of Synset('execute.v.01')>


In [10]:
print(syns[0].examples())

[]


In [11]:
print(syns[0].definition())

lever that activates the firing mechanism of a gun


In [1]:
syns1 = wordnet.synsets("ortho")
print (syns1[0].name())

NameError: name 'wordnet' is not defined

In [19]:
print (syns1[0].definition())

of or relating to orthopedics


In [20]:
print(syns1[0].examples())


['orthopedic shoes']


In [21]:
syns2 = wordnet.synsets("oncology")
print (syns2[0].name())

oncology.n.01


In [22]:
syns2 = wordnet.synsets("oncology")
print (syns2[1].name())

IndexError: list index out of range

In [26]:
from nltk.corpus import semcor
semcor.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [35]:
synonyms = []
for syn in wordnet.synsets("quantitative"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        
print (set(synonyms))

{'quantitative'}


In [33]:
from __future__ import division
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys

# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

######################### word similarity ##########################

def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
               sim = wn.path_similarity(synset_1, synset_2)
               if sim > max_sim:
                   max_sim = sim
                   best_pair = synset_1, synset_2
        return best_pair

def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic 
    ontology (Wordnet in our case as well as the paper's) between two 
    synsets.
    """
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])        
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist)

def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if hypernyms_1.has_key(lcs_candidate):
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if hypernyms_2.has_key(lcs_candidate):
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))
    
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

######################### sentence similarity ##########################

def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
      sim = word_similarity(word, ref_word)
      if sim > max_sim:
          max_sim = sim
          sim_word = ref_word
    return sim_word, max_sim
    
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if not brown_freqs.has_key(word):
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if not brown_freqs.has_key(lookup_word) else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))
    
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec                
            
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))

######################### word order similarity ##########################

def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))

######################### overall similarity ##########################

def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)


sentence_pairs = [
    ["I like that bachelor.", "I like that unmarried man.", 0.561],
    ["John is very nice.", "Is John very nice?", 0.977],
    ["Red alcoholic drink.", "A bottle of wine.", 0.585],
    ["Red alcoholic drink.", "Fresh orange juice.", 0.611],
    ["Red alcoholic drink.", "An English dictionary.", 0.0],
    ["Red alcoholic drink.", "Fresh apple juice.", 0.420],
    ["A glass of cider.", "A full cup of apple juice.", 0.678],
    ["It is a dog.", "That must be your dog.", 0.739],
    ["It is a dog.", "It is a log.", 0.623],
    ["It is a dog.", "It is a pig.", 0.790],
    ["Dogs are animals.", "They are common pets.", 0.738],
    ["Canis familiaris are animals.", "Dogs are common pets.", 0.362],
    ["I have a pen.", "Where do you live?", 0.0],
    ["I have a pen.", "Where is ink?", 0.129],
    ["I have a hammer.", "Take some nails.", 0.508],
    ["I have a hammer.", "Take some apples.", 0.121]
]
for sent_pair in sentence_pairs:
    print ( 
        similarity(sent_pair[0], sent_pair[1], False),
        similarity(sent_pair[0], sent_pair[1], True))
    

TypeError: unorderable types: NoneType() > float()

In [4]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0.0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    score /= count
    return score
 
print (sentence_similarity("He is well","I am suffering"))

TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

In [1]:

from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import wordnet as wn
import numpy as np
from nltk.corpus import brown
import math
import Panda Utils
CONST_PHI = 0.2
CONST_BETA = 0.45
CONST_ALPHA = 0.2
CONST_PHI = 0.2
CONST_DELTA = 0.875
CONST_ETA = 0.4
total_words = 0
word_freq_brown = {}
def proper_synset(word_one , word_two):
    pair = (None,None)
    maximum_similarity = -1
    synsets_one = wn.synsets(word_one)
    synsets_two = wn.synsets(word_two)
    #print("first word :",word_one)
    #print("second word",word_two)
    #print(synsets_one)
    #print(synsets_two)
    if(len(synsets_one)!=0 and len(synsets_two)!=0):
        for synset_one in synsets_one:
            for synset_two in synsets_two:
                similarity = wn.path_similarity(synset_one,synset_two)
                if(similarity == None):
                    sim = -2
                elif(similarity > maximum_similarity):
                    maximum_similarity = similarity
                    pair = synset_one,synset_two
    else:
        #need to see as for some word there will be no wordset.
        #shuld make it as none
        pair = (None , None)
    return pair
def length_between_words(synset_one , synset_two):
    length = 100000000
    if synset_one is None or synset_two is None:
        return 0
    elif(synset_one == synset_two):
        length = 0
    else:
        words_synet1 = set([word.name() for word in synset_one.lemmas()])
        words_synet2 = set([word.name() for word in synset_two.lemmas()])
        if(len(words_synet1) + len(words_synet2) > len(words_synet1.union(words_synet2))):
            length = 0
        else:
            #finding the actual distance
            length = synset_one.shortest_path_distance(synset_two)
            if(length is None):
                return 0
    return math.exp( -1 * CONST_ALPHA * length)
def depth_common_subsumer(synset_one,synset_two):
    height = 100000000
    if synset_one is None or synset_two is None:
        return 0
    elif synset_one == synset_two:
        height = max([hypernym[1] for hypernym in synset_one.hypernym_distances()])
    else:
        #get the hypernym set of both the synset.
        hypernym_one = {hypernym_word[0]:hypernym_word[1] for hypernym_word in synset_one.hypernym_distances()}
        hypernym_two = {hypernym_word[0]:hypernym_word[1] for hypernym_word in synset_two.hypernym_distances()}
        common_subsumer = set(hypernym_one.keys()).intersection(set(hypernym_two.keys()))
        if(len(common_subsumer) == 0):
            height = 0
        else:
            height = 0
            for cs in common_subsumer:
                val = [hypernym_word[1] for hypernym_word in cs.hypernym_distances()]
                val = max(val)
                if val > height : height = val

    #print(height) #works
    return (math.exp(CONST_BETA * height) - math.exp(-CONST_BETA * height))/(math.exp(CONST_BETA * height) + math.exp(-CONST_BETA * height))
def word_similarity(word1,word2):
    #depth_common_subsumer(wn.synset('boy.n.01'),wn.synset('life_form.n.01'))
    #print(wn.synset('boy.n.01').lowest_common_hypernym(wn.synset('animal.n.01')))
    #print(wn.synset('boy.n.01').lowest_common_hypernym(wn.synset('girl.n.01')))
    #word1 = input("Enter the first word: ")
    #word2 = input("Enter the second word: ")
    #synset_wordone = wn.synset(word1+".n.01")#doesnt work
    #synset_wordtwo = wn.synset(word2+".n.01")#doesnt work
    synset_wordone,synset_wordtwo = proper_synset(word1,word2) # cant just add +".n.01" to words to convert them to a synset.
    #Need to execute the above as we cant know whether a 'noun' for of the word exists or not.
    return length_between_words(synset_wordone,synset_wordtwo) * depth_common_subsumer(synset_wordone,synset_wordtwo)

def I(search_word):
    global total_words
    if(total_words == 0):
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in word_freq_brown:
                    word_freq_brown[word] = 0
                word_freq_brown[word] +=1
                total_words+=1
    count = 0 if search_word not in word_freq_brown else word_freq_brown[search_word]
    ret = 1.0 - (math.log(count+1)/math.log(total_words+1))
    return ret
def most_similar_word(word,sentence):
    most_similarity = 0
    most_similar_word = ''
    for w in sentence:
        #compute the word similarity using the already defined function
        sim  =  word_similarity(w,word)
        if sim > most_similarity:
            most_similarity = sim
            most_similar_word = w
    if most_similarity <= CONST_PHI:
        most_similarity = 0
    return most_similar_word,most_similarity 

def gen_sem_vec(sentence , joint_word_set):
    semantic_vector = np.zeros(len(joint_word_set))
    #print(semantic_vector)
    i = 0
    #print("This is sentence :",sentence)
    #print("This is joint word set:",joint_word_set)
    for joint_word in joint_word_set:
        sim_word = joint_word # to measure the 
        beta_sim_measure = 1
        if (joint_word in sentence):
            pass
        else:
            sim_word,beta_sim_measure = most_similar_word(joint_word,sentence) # gets the most similar word in that sentence.
            beta_sim_measure = 0 if beta_sim_measure <= CONST_PHI else beta_sim_measure
        sim_measure = beta_sim_measure * I(joint_word) * I(sim_word)
        #sim_measure = beta_sim_measure ##Without information content which is got from the corpus.
        semantic_vector[i] = sim_measure
        i+=1
    return semantic_vector
def sent_sim(sent_set_one, sent_set_two , joint_word_set):
    #sent_set_one = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_one)))
    #sent_set_two = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_two)))
    #print(sent_set_one)    
    #print(sent_set_two)
    #print(list(sent_set_one.union(sent_set_two)))
    #joint_word_set = list(sent_set_one.union(sent_set_two))
    #print(joint_word_set)
    #sent_set_one = list(sent_set_one)
    #sent_set_two = list(sent_set_two)
    sem_vec_one = gen_sem_vec(sent_set_one,joint_word_set)
    sem_vec_two = gen_sem_vec(sent_set_two,joint_word_set)
    #multiply the two vectors..
    #print(sem_vec_one)
    #print(sem_vec_two)
    return np.dot(sem_vec_one,sem_vec_two.T) / (np.linalg.norm(sem_vec_one) * np.linalg.norm(sem_vec_two))
def word_order_similarity(sentence_one , sentence_two):
    #print("Sentence one :",sentence_one)
    token_one  = word_tokenize(sentence_one)
    #print("Sentence two : ",sentence_two)
    token_two = word_tokenize(sentence_two)
    joint_word_set = list(set(token_one).union(set(token_two)))
    r1 = np.zeros(len(joint_word_set))
    r2 = np.zeros(len(joint_word_set))
    #filling for the first one
    en_joint_one = {x[1]:x[0] for x in enumerate(token_one)}
    en_joint_two = {x[1]:x[0] for x in enumerate(token_two)}
    set_token_one = set(token_one)
    set_token_two = set(token_two)
    i = 0
    #print(en_joint)
    for word in joint_word_set:
        if word in set_token_one:
            r1[i] = en_joint_one[word]#so wrong.
        else:
            #get best word and check if its greater then a preset threshold
            sim_word , sim = most_similar_word(word , list(set_token_one))
            if sim > CONST_ETA : 
                r1[i] = en_joint_one[sim_word]
            else:
                r1[i] = 0
        i+=1
    j = 0
    for word in joint_word_set:
        if word in set_token_two:
            r2[j] = en_joint_two[word]
        else:
            #get best word and check if its greater then a preset threshold
            sim_word , sim = most_similar_word(word , list(set_token_two))
            if sim > CONST_ETA : 
                r2[j] = en_joint_two[sim_word]
            else:
                r2[j] = 0
        j+=1
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))
def main(sentence_one,sentence_two):
    sent_set_one = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_one)))
    sent_set_two = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_two)))
    joint_word_set = list(sent_set_one.union(sent_set_two))
    #Need to get the dictionary to have the corresponding indexes of the joint_word_set. 
    sentence_similarity = (CONST_DELTA * sent_sim(sent_set_one,sent_set_two,list(joint_word_set))) + ((1.0 - CONST_DELTA) * word_order_similarity(sentence_one,sentence_two))
    return sentence_similarity
#sentence_one = "I play hockey"
#sentence_two = "who are you?"
#print(main(sentence_one,sentence_two))\
def file_sem(f):
    contents = open(f).read().strip()
    ind_sentences = sent_tokenize(contents)
    #print(ind_sentences)
    no_of_sentences = len(ind_sentences)
    sent_sim_matr = np.zeros((no_of_sentences,no_of_sentences))
    i = 0
    print(ind_sentences)
    while(i < no_of_sentences):
        j = i
        while(j < no_of_sentences):
            sent_sim_matr[i][j] = main(ind_sentences[i],ind_sentences[j])
            sent_sim_matr[j][i] = sent_sim_matr[i][j]
            j+=1
        i+=1
    return sent_sim_matr
def intro():
    print("\nEnter a valid option:\n")
    print("1.Sentence Similarity between two files containing different sentences.")
    print("2.Sentence similarity between two sentences\n")
    option = int(input("Your choice : "))
    if option == 1:
        file_one = input("Enter the path of the file :")
        #file_two = input("Enter the path of the second file")
        prob_sim_sent = file_sem(file_one)
        print("Similarity between the sentences in a single file is : (IN MATRIX FORM)\n")
        print(prob_sim_sent)
        #could've pickeled , but wrote it to a file.
        f_n = file_one[0:len(file_one)-4:]+"_matrix.txt"
        output_file = open(f_n,'w')
        output_file.write(str(prob_sim_sent))
    elif option == 2:
        sent_one = input("Enter the first sentence : ")
        sent_two = input("Enter the second sentence two :")
        prob_sim_sent = main(sent_one , sent_two)
        print(prob_sim_sent)
        #print("Similarity between\n"+sent_one+"\n"+sent_two+"\n\n is : ",prob_sim_sent)
    else:
        global max_count
        if max_count < 3 : print("Wrong Choice Try again"); max_count+=1 
        else: print("Wrong choice time exceeded!");exit()
        intro()
if __name__ == "__main__":  
    print("-------------------Sentence Similarity--------------------------")
    intro()
    print("Want to try once again? if yes press 1 or else 0")
    excited = int(input())
    while(excited == 1):
        intro()
        print("Want to try once again?")
        excited = int(input())

-------------------Sentence Similarity--------------------------

Enter a valid option:

1.Sentence Similarity between two files containing different sentences.
2.Sentence similarity between two sentences

Your choice : 1
Enter the path of the file :1


FileNotFoundError: [Errno 2] No such file or directory: '1'

In [9]:
!python 

  Using cached https://files.pythonhosted.org/packages/46/dc/7fd5df840efb3e56c8b4f768793a237ec4ee59891959d6a215d63f727023/pip-19.0.1-py2.py3-none-any.whl


  Could not find a version that satisfies the requirement user (from versions: )
No matching distribution found for user
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import os

import pandas as pd
import requests
import tensorflow as tf


def load_sts_dataset(filename):
    """
     Loads a subset of the STS dataset into a DataFrame.
     In particular both sentences and their human rated similarity score.
    :param filename:
    :return:
    """
    sent_pairs = []
    with tf.gfile.GFile(filename, "r") as f:
        for line in f:
            ts = line.strip().split("\t")
            sent_pairs.append((ts[5], ts[6], float(ts[4])))
    return pd.DataFrame(sent_pairs, columns=["sent_1", "sent_2", "sim"])


def download_and_load_sts_data():
    sts_dataset = tf.keras.utils.get_file(
        fname="mesh.nt.gz",
        origin="ftp://ftp.nlm.nih.gov/online/mesh/rdf/mesh.nt.gz",
        extract=True)

    sts_dev = load_sts_dataset(os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"))
    sts_test = load_sts_dataset(os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"))

    return sts_dev, sts_test


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:


import nltk
from nltk import word_tokenize
import pandas as pd


def tokenize(q1, q2):
    """
        q1 and q2 are sentences/questions. Function returns a list of tokens for both.
    """
    return word_tokenize(q1), word_tokenize(q2)


def posTag(q1, q2):
    """
        q1 and q2 are lists. Function returns a list of POS tagged tokens for both.
    """
    return nltk.pos_tag(q1), nltk.pos_tag(q2)


def stemmer(tag_q1, tag_q2):
    """
        tag_q = tagged lists. Function returns a stemmed list.
    """

    stem_q1 = []
    stem_q2 = []

    for token in tag_q1:
        stem_q1.append(stem(token))

    for token in tag_q2:
        stem_q2.append(stem(token))

    return stem_q1, stem_q2

def intro():
    print("\nEnter a valid option:\n")
    print("1.Sentence Similarity between two files containing different sentences.")
    print("2.Sentence similarity between two sentences\n")
    option = int(input("Your choice : "))
    if option == 1:
        file_one = input("Enter the path of the file :")
        #file_two = input("Enter the path of the second file")
        prob_sim_sent = word_tokenize(file_one)
        print("Similarity between the sentences in a single file is : (IN MATRIX FORM)\n")
        print(prob_sim_sent)
        #could've pickeled , but wrote it to a file.
        f_n = file_one[0:len(file_one)-4:]+"_matrix.txt"
        output_file = open(f_n,'w')
        output_file.write(str(prob_sim_sent))
    elif option == 2:
        sent_one = input("Enter the first sentence : ")
        sent_two = input("Enter the second sentence two :")
        prob_sim_sent = tokenize(sent_one , sent_two)
        print(prob_sim_sent)
        #print("Similarity between\n"+sent_one+"\n"+sent_two+"\n\n is : ",prob_sim_sent)
    else:
        global max_count
        if max_count < 3 : print("Wrong Choice Try again"); max_count+=1 
        else: print("Wrong choice time exceeded!");exit()
        intro()
        
if __name__ == "__main__":  
    print("-------------------Sentence Similarity--------------------------")
    intro()
    print("Want to try once again? if yes press 1 or else 0")
    excited = int(input())
    while(excited == 1):
        intro()
        print("Want to try once again?")
        excited = int(input())

-------------------Sentence Similarity--------------------------

Enter a valid option:

1.Sentence Similarity between two files containing different sentences.
2.Sentence similarity between two sentences

Your choice : 2
Enter the first sentence : hi this is ravi
Enter the second sentence two :hi whoese is ravi
(['hi', 'this', 'is', 'ravi'], ['hi', 'whoese', 'is', 'ravi'])
Want to try once again? if yes press 1 or else 0


In [24]:

from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import wordnet as wn
import numpy as np
from nltk.corpus import brown
import math
import pandas as pd
CONST_PHI = 0.2
CONST_BETA = 0.45
CONST_ALPHA = 0.2
CONST_PHI = 0.2
CONST_DELTA = 0.875
CONST_ETA = 0.4
total_words = 0
word_freq_brown = {}

def Get_NewCodes():
    df = pd.read_csv("Same_SIM_Draft.csv")
    sCodes = df['LongDesc'].tolist()
    return sCodes; 
        
def Get_MasterCodeDesc():
    df = pd.read_csv("Same_SIM_Master_Draft.csv")
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;


def proper_synset(word_one , word_two):
    pair = (None,None)
    maximum_similarity = -1
    synsets_one = wn.synsets(word_one)
    synsets_two = wn.synsets(word_two)
    #print("first word :",word_one)
    #print("second word",word_two)
    #print(synsets_one)
    #print(synsets_two)
    if(len(synsets_one)!=0 and len(synsets_two)!=0):
        for synset_one in synsets_one:
            for synset_two in synsets_two:
                similarity = wn.path_similarity(synset_one,synset_two)
                if(similarity == None):
                    sim = -2
                elif(similarity > maximum_similarity):
                    maximum_similarity = similarity
                    pair = synset_one,synset_two
    else:
        #need to see as for some word there will be no wordset.
        #shuld make it as none
        pair = (None , None)
    return pair
def length_between_words(synset_one , synset_two):
    length = 100000000
    if synset_one is None or synset_two is None:
        return 0
    elif(synset_one == synset_two):
        length = 0
    else:
        words_synet1 = set([word.name() for word in synset_one.lemmas()])
        words_synet2 = set([word.name() for word in synset_two.lemmas()])
        if(len(words_synet1) + len(words_synet2) > len(words_synet1.union(words_synet2))):
            length = 0
        else:
            #finding the actual distance
            length = synset_one.shortest_path_distance(synset_two)
            if(length is None):
                return 0
    return math.exp( -1 * CONST_ALPHA * length)
def depth_common_subsumer(synset_one,synset_two):
    height = 100000000
    if synset_one is None or synset_two is None:
        return 0
    elif synset_one == synset_two:
        height = max([hypernym[1] for hypernym in synset_one.hypernym_distances()])
    else:
        #get the hypernym set of both the synset.
        hypernym_one = {hypernym_word[0]:hypernym_word[1] for hypernym_word in synset_one.hypernym_distances()}
        hypernym_two = {hypernym_word[0]:hypernym_word[1] for hypernym_word in synset_two.hypernym_distances()}
        common_subsumer = set(hypernym_one.keys()).intersection(set(hypernym_two.keys()))
        if(len(common_subsumer) == 0):
            height = 0
        else:
            height = 0
            for cs in common_subsumer:
                val = [hypernym_word[1] for hypernym_word in cs.hypernym_distances()]
                val = max(val)
                if val > height : height = val

    #print(height) #works
    return (math.exp(CONST_BETA * height) - math.exp(-CONST_BETA * height))/(math.exp(CONST_BETA * height) + math.exp(-CONST_BETA * height))
def word_similarity(word1,word2):
    #depth_common_subsumer(wn.synset('boy.n.01'),wn.synset('life_form.n.01'))
    #print(wn.synset('boy.n.01').lowest_common_hypernym(wn.synset('animal.n.01')))
    #print(wn.synset('boy.n.01').lowest_common_hypernym(wn.synset('girl.n.01')))
    #word1 = input("Enter the first word: ")
    #word2 = input("Enter the second word: ")
    #synset_wordone = wn.synset(word1+".n.01")#doesnt work
    #synset_wordtwo = wn.synset(word2+".n.01")#doesnt work
    synset_wordone,synset_wordtwo = proper_synset(word1,word2) # cant just add +".n.01" to words to convert them to a synset.
    #Need to execute the above as we cant know whether a 'noun' for of the word exists or not.
    return length_between_words(synset_wordone,synset_wordtwo) * depth_common_subsumer(synset_wordone,synset_wordtwo)

def I(search_word):
    global total_words
    if(total_words == 0):
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in word_freq_brown:
                    word_freq_brown[word] = 0
                word_freq_brown[word] +=1
                total_words+=1
    count = 0 if search_word not in word_freq_brown else word_freq_brown[search_word]
    ret = 1.0 - (math.log(count+1)/math.log(total_words+1))
    return ret
def most_similar_word(word,sentence):
    most_similarity = 0
    most_similar_word = ''
    for w in sentence:
        #compute the word similarity using the already defined function
        sim  =  word_similarity(w,word)
        if sim > most_similarity:
            most_similarity = sim
            most_similar_word = w
    if most_similarity <= CONST_PHI:
        most_similarity = 0
    return most_similar_word,most_similarity 

def gen_sem_vec(sentence , joint_word_set):
    semantic_vector = np.zeros(len(joint_word_set))
    #print(semantic_vector)
    i = 0
    #print("This is sentence :",sentence)
    #print("This is joint word set:",joint_word_set)
    for joint_word in joint_word_set:
        sim_word = joint_word # to measure the 
        beta_sim_measure = 1
        if (joint_word in sentence):
            pass
        else:
            sim_word,beta_sim_measure = most_similar_word(joint_word,sentence) # gets the most similar word in that sentence.
            beta_sim_measure = 0 if beta_sim_measure <= CONST_PHI else beta_sim_measure
        sim_measure = beta_sim_measure * I(joint_word) * I(sim_word)
        #sim_measure = beta_sim_measure ##Without information content which is got from the corpus.
        semantic_vector[i] = sim_measure
        i+=1
    return semantic_vector
def sent_sim(sent_set_one, sent_set_two , joint_word_set):
    #sent_set_one = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_one)))
    #sent_set_two = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_two)))
    #print(sent_set_one)    
    #print(sent_set_two)
    #print(list(sent_set_one.union(sent_set_two)))
    #joint_word_set = list(sent_set_one.union(sent_set_two))
    #print(joint_word_set)
    #sent_set_one = list(sent_set_one)
    #sent_set_two = list(sent_set_two)
    sem_vec_one = gen_sem_vec(sent_set_one,joint_word_set)
    sem_vec_two = gen_sem_vec(sent_set_two,joint_word_set)
    #multiply the two vectors..
    #print(sem_vec_one)
    #print(sem_vec_two)
    return np.dot(sem_vec_one,sem_vec_two.T) / (np.linalg.norm(sem_vec_one) * np.linalg.norm(sem_vec_two))
def word_order_similarity(sentence_one , sentence_two):
    #print("Sentence one :",sentence_one)
    token_one  = word_tokenize(sentence_one)
    #print("Sentence two : ",sentence_two)
    token_two = word_tokenize(sentence_two)
    joint_word_set = list(set(token_one).union(set(token_two)))
    r1 = np.zeros(len(joint_word_set))
    r2 = np.zeros(len(joint_word_set))
    #filling for the first one
    en_joint_one = {x[1]:x[0] for x in enumerate(token_one)}
    en_joint_two = {x[1]:x[0] for x in enumerate(token_two)}
    set_token_one = set(token_one)
    set_token_two = set(token_two)
    i = 0
    #print(en_joint)
    for word in joint_word_set:
        if word in set_token_one:
            r1[i] = en_joint_one[word]#so wrong.
        else:
            #get best word and check if its greater then a preset threshold
            sim_word , sim = most_similar_word(word , list(set_token_one))
            if sim > CONST_ETA : 
                r1[i] = en_joint_one[sim_word]
            else:
                r1[i] = 0
        i+=1
    j = 0
    for word in joint_word_set:
        if word in set_token_two:
            r2[j] = en_joint_two[word]
        else:
            #get best word and check if its greater then a preset threshold
            sim_word , sim = most_similar_word(word , list(set_token_two))
            if sim > CONST_ETA : 
                r2[j] = en_joint_two[sim_word]
            else:
                r2[j] = 0
        j+=1
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))
def main(sentence_one,sentence_two):
    sent_set_one = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_one)))
    sent_set_two = set(filter(lambda x : not (x == '.' or x == '?') , word_tokenize(sentence_two)))
    joint_word_set = list(sent_set_one.union(sent_set_two))
    #Need to get the dictionary to have the corresponding indexes of the joint_word_set. 
    sentence_similarity = (CONST_DELTA * sent_sim(sent_set_one,sent_set_two,list(joint_word_set))) + ((1.0 - CONST_DELTA) * word_order_similarity(sentence_one,sentence_two))
    return sentence_similarity
#sentence_one = "I play hockey"
#sentence_two = "who are you?"
#print(main(sentence_one,sentence_two))\
def file_sem(f):
    contents = open(f).read().strip()
    ind_sentences = sent_tokenize(contents)
    #print(ind_sentences)
    no_of_sentences = len(ind_sentences)
    sent_sim_matr = np.zeros((no_of_sentences,no_of_sentences))
    i = 0
    print(ind_sentences)
    while(i < no_of_sentences):
        j = i
        while(j < no_of_sentences):
            sent_sim_matr[i][j] = main(ind_sentences[i],ind_sentences[j])
            sent_sim_matr[j][i] = sent_sim_matr[i][j]
            j+=1
        i+=1
    return sent_sim_matr

def intro():
    sent_one = Get_NewCodes()
    sent_two = Get_MasterCodeDesc()
    for iNewLongDesc in sent_one:  
            for iMasterDesc in sent_two:
                    prob_sim_sent = main(iNewLongDesc , iMasterDesc)
                    print(prob_sim_sent)
    

if __name__ == "__main__":
        intro()
     


0.32989370842
0.392684764655
0.215971016507
0.309323112201
0.364590693528
0.378395628396


In [11]:
import gzip
import gensim 
import logging
import os
import numpy as np
from gensim.models import Word2Vec
from scipy import spatial

def show_file_contents(input_file):
    with gzip.open(input_file, 'rb') as f:
        for i, line in enumerate(f):
            print(line)
            if (i==1000):
                print("reached 100")
                print("read {0} reviews".format(i))
            break
        
    yield gensim.utils.simple_preprocess(line)
            
                
                
 
def read_input(input_file):
    
    print("method ivoked")
    """This method reads the input file which is in gzip format"""
    with gzip.open(input_file, 'rb') as f:
        print ("opened file")
        for i, line in enumerate(f):

            if (i % 10 == 0):
                #print("getting dedauks")
                print("read {0} reviews".format(i))
            # do some pre-processing and return list of words for each review
            # text
            yield gensim.utils.simple_preprocess(line)

index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


if __name__ == "__main__":

        #abspath = os.path.dirname(os.path.abspath(__file__))
        data_file = os.path.join("Same_SIM_Master_List.csv.gz")

    # read the tokenized reviews into a list
    # each review item becomes a serries of words
    # so this becomes a list of lists
        documents = list(read_input(data_file))
        #print(documents )
        model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=1, workers=10)
        model.train(documents,total_examples=len(documents),epochs=10)
        w1 = "oncology"
        print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))
        
        s1_afv = avg_feature_vector('oncology (breast); mrna; gene expression profiling by real-time rt-pcr of 21 genes; utilizing formalin-fixed paraffin-embedded tissue; algorithm reported as recurrence score', model=model, num_features=150, index2word_set=index2word_set)
        s2_afv = avg_feature_vector('oncology (breast); mrna; gene expression profiling by real-time rt-pcr of 21 genes; utilizing formalin-fixed paraffin-embedded tissue; algorithm reported as recurrence score', model=model, num_features=150, index2word_set=index2word_set)
        sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
        print(sim)
      

method ivoked
opened file
read 0 reviews
read 10 reviews
read 20 reviews
read 30 reviews
read 40 reviews
read 50 reviews
read 60 reviews
read 70 reviews
read 80 reviews
read 90 reviews
read 100 reviews
read 110 reviews
read 120 reviews
read 130 reviews
read 140 reviews
read 150 reviews
read 160 reviews
read 170 reviews
read 180 reviews
read 190 reviews
read 200 reviews
read 210 reviews
read 220 reviews
read 230 reviews
read 240 reviews
read 250 reviews
read 260 reviews
read 270 reviews
read 280 reviews
read 290 reviews
read 300 reviews
read 310 reviews
read 320 reviews
read 330 reviews
read 340 reviews
read 350 reviews
read 360 reviews
read 370 reviews
read 380 reviews
read 390 reviews
read 400 reviews
read 410 reviews
read 420 reviews
read 430 reviews
read 440 reviews
read 450 reviews
read 460 reviews
read 470 reviews
read 480 reviews
read 490 reviews
read 500 reviews
read 510 reviews
read 520 reviews
read 530 reviews
read 540 reviews
read 550 reviews
read 560 reviews
read 570 reviews

read 4800 reviews
read 4810 reviews
read 4820 reviews
read 4830 reviews
read 4840 reviews
read 4850 reviews
read 4860 reviews
read 4870 reviews
read 4880 reviews
read 4890 reviews
read 4900 reviews
read 4910 reviews
read 4920 reviews
read 4930 reviews
read 4940 reviews
read 4950 reviews
read 4960 reviews
read 4970 reviews
read 4980 reviews
read 4990 reviews
read 5000 reviews
read 5010 reviews
read 5020 reviews
read 5030 reviews
read 5040 reviews
read 5050 reviews
read 5060 reviews
read 5070 reviews
read 5080 reviews
read 5090 reviews
read 5100 reviews
read 5110 reviews
read 5120 reviews
read 5130 reviews
read 5140 reviews
read 5150 reviews
read 5160 reviews
read 5170 reviews
read 5180 reviews
read 5190 reviews
read 5200 reviews
read 5210 reviews
read 5220 reviews
read 5230 reviews
read 5240 reviews
read 5250 reviews
read 5260 reviews
read 5270 reviews
read 5280 reviews
read 5290 reviews
read 5300 reviews
read 5310 reviews
read 5320 reviews
read 5330 reviews
read 5340 reviews
read 5350 

read 12480 reviews
read 12490 reviews
read 12500 reviews
read 12510 reviews
read 12520 reviews
read 12530 reviews
read 12540 reviews
read 12550 reviews
read 12560 reviews
read 12570 reviews
read 12580 reviews
read 12590 reviews
read 12600 reviews
read 12610 reviews
read 12620 reviews
read 12630 reviews
read 12640 reviews
read 12650 reviews
read 12660 reviews
read 12670 reviews
read 12680 reviews
read 12690 reviews
read 12700 reviews
read 12710 reviews
read 12720 reviews
read 12730 reviews
read 12740 reviews
read 12750 reviews
read 12760 reviews
read 12770 reviews
read 12780 reviews
read 12790 reviews
read 12800 reviews
read 12810 reviews
read 12820 reviews
read 12830 reviews
read 12840 reviews
read 12850 reviews
read 12860 reviews
read 12870 reviews
read 12880 reviews
read 12890 reviews
read 12900 reviews
read 12910 reviews
read 12920 reviews
read 12930 reviews
read 12940 reviews
read 12950 reviews
read 12960 reviews
read 12970 reviews
read 12980 reviews
read 12990 reviews
read 13000 r

read 18730 reviews
read 18740 reviews
read 18750 reviews
read 18760 reviews
read 18770 reviews
read 18780 reviews
read 18790 reviews
read 18800 reviews
read 18810 reviews
read 18820 reviews
read 18830 reviews
read 18840 reviews
read 18850 reviews
read 18860 reviews
read 18870 reviews
read 18880 reviews
read 18890 reviews
read 18900 reviews
read 18910 reviews
read 18920 reviews
read 18930 reviews
read 18940 reviews
read 18950 reviews
read 18960 reviews
read 18970 reviews
read 18980 reviews
read 18990 reviews
read 19000 reviews
read 19010 reviews
read 19020 reviews
read 19030 reviews
read 19040 reviews
read 19050 reviews
read 19060 reviews
read 19070 reviews
read 19080 reviews
read 19090 reviews
read 19100 reviews
read 19110 reviews
read 19120 reviews
read 19130 reviews
read 19140 reviews
read 19150 reviews
read 19160 reviews
read 19170 reviews
read 19180 reviews
read 19190 reviews
read 19200 reviews
read 19210 reviews
read 19220 reviews
read 19230 reviews
read 19240 reviews
read 19250 r

Most similar to oncology [('allelic', 0.8633838295936584), ('imbalance', 0.8616378307342529), ('myelogenous', 0.84666907787323), ('mrna', 0.841934323310852), ('expression', 0.8412970900535583), ('hereditary', 0.8280978202819824), ('urothelial', 0.8223068118095398), ('cancer', 0.8216057419776917), ('colorectal', 0.8202099800109863), ('gstp', 0.8124186396598816)]
1.0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [1]:
import gzip
import gensim 
import logging
import os
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def read_input(input_file):
    
    print("method ivoked")
    """This method reads the input file which is in gzip format"""
    with gzip.open(input_file, 'rb') as f:
        print ("opened file")
        for i, line in enumerate(f):

            if (i % 10 == 0):
                #print("getting dedauks")
                "read {0} reviews".format(i)
            # do some pre-processing and return list of words for each review
            # text
            yield gensim.utils.simple_preprocess(line)

data_file = os.path.join("Same_SIM_Master_List.csv.gz")
data = list(read_input(data_file))
print(len(data))
#data = ["oncology (breast); mrna; gene expression profiling by real-time rt-pcr of 21 genes; utilizing formalin-fixed paraffin-embedded tissue; algorithm reported as recurrence score",
        #"flt3 (fms-related tyrosine kinase 3) (eg; acute myeloid leukemia); gene analysis; internal tandem duplication (itd) variants (ie; exons 14; 15)",
        #"npm1 (nucleophosmin) (eg; acute myeloid leukemia) gene analysis; quantitative",
        #"oncology (colon); mrna; gene expression profiling by real-time rt-pcr of 12 genes (7 content and 5 housekeeping); utilizing formalin-fixed paraffin-embedded tissue; algorithm reported as a recurrence score","they chat really well","he is running out of time for chatting","i am having cup of tea"]

for idata in data:   
    tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(idata)]

    
print(tagged_data)
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d1v.model")
print("Model Saved")

method ivoked
opened file
24827
[TaggedDocument(words=['dysphagia'], tags=['0']), TaggedDocument(words=['screening'], tags=['1'])]
iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:579: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).



iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteration 78
iteration 79
iteration 80
iteration 81
iteration 82
iteration 83
iteration 84
iteration 85
iteration 86
iteration 87
iteration 88
iteration 89
iteration 90
iteration 91
iteration 92
iteration 93
iteration 94
iteration 95
iteration 9

In [2]:
model= Doc2Vec.load("d1v.model")

In [3]:

tokens = word_tokenize("dysphagia page")

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar(positive=[model.infer_vector(tokens)],topn=3)
print (sims)

[('0', 0.26953816413879395), ('1', -0.2932582497596741)]


In [16]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# random
from random import shuffle

# classifier
from sklearn.linear_model import LogisticRegression

class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences
    
sources = {'Same_SIM_Master_List.csv':'TEST_NEG'}

sentences = LabeledLineSentence(sources)

sent = sentences.to_array()

print(len(sent))

model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)

model.build_vocab(sentences.to_array())

for epoch in range(100):
    print('iteration {0}'.format(epoch))
    model.train(sentences.to_array(),
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


24827


C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:579: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [17]:
model.save("Lable.model")
print("Model Saved")

Model Saved


In [37]:
from nltk.tokenize import word_tokenize

model = Doc2Vec.load('Lable.model')
tokens = word_tokenize("oncology Breast")
new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar(positive=[model.infer_vector(tokens)],topn=5)
print (sims)

print(TEST_NEG[12203])

[('TEST_NEG_17203', 0.8862501978874207), ('TEST_NEG_8575', 0.8798203468322754), ('TEST_NEG_4575', 0.8773929476737976), ('TEST_NEG_16888', 0.8762221932411194), ('TEST_NEG_17563', 0.8737908601760864)]


NameError: name 'TEST_NEG' is not defined

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd

mylist = list()
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy'),index=range(60000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
    
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv("Same_SIM_Master_List.csv")
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv("NewCodes.csv")
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv("NewCodes.csv")
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,x):
    df = pd.read_csv("Same_SIM_Master_List.csv")    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)
    tokenizer = TfidfVectorizer(stop_words = 'english') # initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
    score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
    #score
    pd.set_option("display.max_rows", 11)
    sortedscore = score.sort_values(by=['score'], ascending=False)
    sortedscore.head(11)
    for x in range(0,10):
        rownum = rownum+1
        svalue = sortedscore.iloc[x,1]
        Accuracy = sortedscore.iloc[x,2]
        result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,rownum)
 
resultFrame

export_csv = resultFrame.to_csv ('export_dataframe3.csv', index = None, header=True)

